In this notebook we explore the effect of weather in influencing the visitor numbers.

We make use of Fixed effects approach to correctly account (control) for static infleunces while exploring the effect of the dynamic variables.

https://matheusfacure.github.io/python-causality-handbook/14-Panel-Data-and-Fixed-Effects.html

In [ ]:
os.chdir('..')

In [ ]:
%load_ext autoreload
%autoreload 2

# All the variables are defined in the Config file
from model_config import *
from model_packages import *
from model_utils import *

In [ ]:
data_counter_strava=pd.read_pickle(data_folder+'strava_data_all_sites.pkl')

weather_df_all_sites=pd.read_pickle(data_folder+'weather_df.pkl')

weather_df_all_sites['Date']=weather_df_all_sites['Date'].astype(str)

data_counter_strava.dropna().groupby('Date')['site'].count().plot(style='-o')
plt.ylabel('Number of monitoring sites')

In [ ]:
data_counter_strava.merge(weather_df_all_sites,on=['Date','site'],how='inner').groupby('Date')['site'].count().plot(style='-o')
plt.ylabel('Number of monitoring sites with feature list available')

In [ ]:
data_with_training_sites=data_counter_strava.merge(weather_df_all_sites,on=['Date','site'],how='inner').dropna()

data_with_training_sites['year']=data_with_training_sites['Date'].apply(lambda x: int(x.split('-')[0]))

data_with_training_sites['Date']=data_with_training_sites['Date'].apply(lambda x: int(x.split('-')[1]))


data_with_training_sites['season']=data_with_training_sites['Date'].apply(lambda x : get_season(x))


data_with_training_sites=data_with_training_sites.sort_values(by=['site']).reset_index(drop=True)

data_with_training_sites.rename(columns={'Date':'Month'},inplace=True)

data_with_training_sites['Month']=data_with_training_sites['Month'].map(dict(zip(range(1,13),\
                                                                           [calendar.month_abbr[x]\
                                                                            for x in range(1,13)])))

data_with_training_sites.sample(5)

In [ ]:
data=data_with_training_sites.copy()

In [ ]:
site_names=[x for x in data['site'].unique()]

In [ ]:
# Effect of year:

# Regressing against a dummy variable (the only predictor) 
# is the same as getting mean of the output
# variable grouped by that dummy variable.

In [ ]:
# Ordinary least square approach-neglects the panel data
# structure of the data (repeated observations)
mod = smf.ols("people_counter_data ~ C(year)", data=data).fit()
print(mod.summary().tables[1])

# Get the mean grouped by year
data.groupby("year")["people_counter_data"].mean()

In [ ]:
# Effect of year:

mod = smf.ols("people_counter_data ~ C(season)", data=data).fit()
print(mod.summary().tables[1])

data.groupby("season")["people_counter_data"].mean()

In [ ]:
# variables which are not-static
#data.groupby("site").std().sum()

In [ ]:
data=data[['site','people_counter_data','tavg','year','total_trip_count','season','Month']]

data

In [ ]:
# Fixed effects approach 

# https://matheusfacure.github.io/python-causality-handbook/14-Panel-Data-and-Fixed-Effects.html

Y = "people_counter_data" # outcome variable
T = "total_trip_count" # treatment
X = [T]+['tavg'] #list of predictors

In [ ]:
# Get the mean observation for each site:
mean_data = data.groupby(["site"])[X+[Y]].mean()

#de-mean data

demeaned_data = (data
               .set_index(["site"]) # set the index as the site indicator
               [X+[Y]]
               - mean_data) # subtract the mean data

# ols on de-mean data- this way we control for all fixed static influences
mod = smf.ols(f"{Y} ~ {'+'.join(X)}", data=demeaned_data).fit()
mod.summary().tables[1]

In [ ]:
# Panel-data out of box approach: identical results to the above result
mod = PanelOLS.from_formula(f"{Y} ~ {'+'.join(X)}"+'+ EntityEffects',data=data.set_index(["site","year"]))

result = mod.fit(cov_type='clustered', cluster_entity=True)
result.summary.tables[1]

In [ ]:
# If adding a dummy for each individual controls for fixed site characteristics, 
# adding a time dummy would control for variables that are fixed for each time period,
# but that might change across time. e.g. covid restrictions
mod = PanelOLS.from_formula(f"{Y} ~ {'+'.join(X)}"+'+ EntityEffects+TimeEffects',
                            data=data.set_index(["site","year"]),drop_absorbed=True)

result = mod.fit(cov_type='clustered', cluster_entity=True)
result.summary.tables[1]

In [ ]:
# Ordinary-least square approach-has potential bias because
# of omitted variables

mod = smf.ols("people_counter_data ~ tavg+total_trip_count", data=data).fit()
mod.summary().tables[1]

In [ ]:
# in-sample predictions

In [ ]:
predtnc=result.predict(fitted=True,effects=True,idiosyncratic=True).reset_index()

predtnc['Month']=data['Month'].map(dict(zip([calendar.month_abbr[x] for x in range(1,13)],range(1,13))))

predtnc['Date']=predtnc['Month'].astype(str)+'-'+predtnc['year'].astype(str)

predtnc['people_counter_data']=data['people_counter_data']

predtnc['Date']=pd.to_datetime(predtnc['Date'])#

predtnc['season']=data['season']

model_actls=predtnc.groupby(['Date'])['people_counter_data'].mean().plot(label='Counter data')
model_predctns=predtnc.groupby(['Date'])['fitted_values'].mean().plot(label='In sample predictions')
plt.legend()


predtnc=predtnc.groupby(['season','site'])[['people_counter_data','fitted_values']].mean().reset_index()
#Visualisation of the time series data

In [ ]:
# Add counter locations

sites_df=gpd.read_file(data_folder+'accessibility.shp')

sites_df=sites_df[sites_df['geom_type']=='5km buffer'].reset_index(drop=True)



predtnc=predtnc.merge(sites_df,left_on=['site'],right_on=['counter'],how='inner')\





predtnc.geometry=gpd.GeoDataFrame(predtnc).centroid.to_crs(crs_deg)

predtnc['latitude'] = predtnc.geometry.apply(lambda p: p.y)
predtnc['longitude'] =predtnc.geometry.apply(lambda p: p.x)

In [ ]:
# heat-map for a specific season

predtnc_sesn=predtnc[predtnc['season']=='summer']

In [ ]:
#Actual
fig = px.density_mapbox(predtnc_sesn, lat='latitude', lon='longitude', z='people_counter_data',
                        mapbox_style="stamen-terrain")
 
fig

In [ ]:
#Prediction
fig = px.density_mapbox(predtnc_sesn, lat='latitude', lon='longitude', z='fitted_values',
                        mapbox_style="stamen-terrain")
 
fig

In [ ]:
# Controlling for seasons:

for sesn in list(data['season'].unique()):
    #data_sesn=data[data['season']==sesn]
    scaler = RobustScaler()
    
    data_sesn=data[data['season']==sesn].reset_index(drop=True)
    
    data_sesn_scln=data_sesn[X+[Y]].copy()
    
    cols=data_sesn_scln.columns
    
    
    # we are developing a separate model for each season, so we need to standardise the coefs
    data_sesn_scln=scaler.fit_transform(data_sesn_scln)
    
    data_sesn_scln=pd.DataFrame(data_sesn_scln,columns=cols)
    
    
    
    data_sesn[X+[Y]]=data_sesn_scln
    
    print('Fixed effects model for {}'.format(sesn))
    #mod = PanelOLS.from_formula(f"{Y} ~ {'+'.join(X)}"+'+ EntityEffects',
    #                        data=data_sesn.set_index(["site","year"]),drop_absorbed=True)
    #result = mod.fit(cov_type='clustered', cluster_entity=True)
    
    
    #print(result.summary.tables[1])
    
    
    #from linearmodels.panel import PanelOLS
    
    mean_data = data_sesn.groupby(["site"])[X+[Y]].mean()
    
    demeaned_data = (data_sesn
               .set_index(["site"]) # set the index as the site indicator
               [X+[Y]]
               - mean_data) # subtract the mean data
    
    mod = smf.ols(f"{Y} ~ {'+'.join(X)}", data=demeaned_data).fit()
    
    print(mod.summary().tables[1])
    print('+'*100)

In [ ]:
# Looking at the spatial variation in 
# the effect of weather/strava count 

data_sites=data.reset_index()
# Evaluating the influence of weather for each site

concat_coef=[]

for indv_site in list(data_sites['site'].unique()):
    
    
    scaler = StandardScaler()
    data_site=data_sites[data_sites['site']==indv_site]
    
    data_site=data_site[X+[Y]]
    
    cols=data_site.columns
    
    # standardise the data if we have to compare regression coeffs 
    # across mutliple sites
    data_site=scaler.fit_transform(data_site)
    
    data_site=pd.DataFrame(data_site,columns=cols)
    
    X_regr,y_regr=data_site[X].copy(), data_site[Y].copy()
    
    # Getting regression coefficients as table
    lm = pg.linear_regression(pd.DataFrame(X_regr,columns=X), y_regr)
    
    lm['site']=indv_site
    
    concat_coef.append(lm)

In [ ]:
var='total_trip_count'#'tavg'#

coef_all_sites=pd.concat(concat_coef)
coef_all_sites=coef_all_sites[coef_all_sites['names']==var].sort_values(by='coef')

# only the non-significant coefs
coef_all_sites_ns=coef_all_sites[coef_all_sites['pval']>=0.1]

# only the significant coefs
coef_all_sites=coef_all_sites[coef_all_sites['pval']<0.1]

coef_all_sites=coef_all_sites[coef_all_sites['coef']>0]

In [ ]:
# regression coefs
coef_all_sites.set_index('site').sort_values(by='coef')['coef'].plot(kind='barh',figsize=(10,5))
plt.xlabel('{} (Regression cofficient)'.format(var))

In [ ]:
# Spatial location of the sites

sites_df=gpd.read_file(data_folder+'accessibility.shp')

sites_df=sites_df[sites_df['geom_type']=='5km buffer'].reset_index(drop=True)



coef_all_sites_geo=coef_all_sites.set_index('site').sort_values(by='coef').reset_index().merge(sites_df,left_on=['site'],\
right_on=['counter'],how='inner')[['site','coef','pval','geometry']]




coef_all_sites_geo.geometry=gpd.GeoDataFrame(coef_all_sites_geo).centroid.to_crs(crs_deg)

coef_all_sites_geo['latitude'] = coef_all_sites_geo.geometry.apply(lambda p: p.y)
coef_all_sites_geo['longitude'] = coef_all_sites_geo.geometry.apply(lambda p: p.x)

#coef_all_sites_geo['abs_coef']=np.abs(coef_all_sites_geo['coef'])

# Visualisation of mean dog occupancy and number of visits.
fig = px.scatter_mapbox(coef_all_sites_geo, lat="latitude", lon="longitude",\
                        color="coef", size="coef",
                        color_continuous_scale="RdYlGn_r",
                        center={"lat": coef_all_sites_geo['latitude'].mean(),\
                                "lon": coef_all_sites_geo['longitude'].mean()}, zoom=3.5,
                        mapbox_style="carto-positron", hover_name="site")
fig.update_layout(margin=dict(l=0, r=0, t=30, b=10))
fig.show()

In [ ]:
# k-means clustering on the sites

df=sites_df.copy()
df.geometry=df.geometry.centroid




df['lon'] = df.geometry.apply(lambda p: p.x)
df['lat'] = df.geometry.apply(lambda p: p.y)


sites_geom_tvg=df.merge(coef_all_sites_geo[['site','coef']],left_on=['counter'],right_on=['site'],how='inner')

df=sites_geom_tvg

In [ ]:
# Find the optimal cluster

K_clusters = range(1,10)
kmeans = [KMeans(n_clusters=i) for i in K_clusters]
Y_axis = df[['lat']]
X_axis = df[['lon']]
score = [kmeans[i].fit(Y_axis).score(Y_axis) for i in range(len(kmeans))]
# Visualize
plt.plot(K_clusters, score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()


K_clusters = range(1,10)
kmeans = [KMeans(n_clusters=i) for i in K_clusters]
lat_long = df[['lat','lon']]
lot_size = df['coef']
sample_weight = lot_size
score = [kmeans[i].fit(lat_long, sample_weight = lot_size).score(lat_long) for i in range(len(kmeans))]
plt.plot(K_clusters, score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve = Weighted')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 2, init ='k-means++')
kmeans.fit(df[['lon','lat']]) # Compute k-means clustering.
df['cluster_label'] = kmeans.fit_predict(df[['lon','lat']])
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels = kmeans.predict(df[['lon','lat']]) # Labels of each point
ax=df.plot.scatter(y = 'lat', x = 'lon', c=labels, s=50, cmap='viridis')
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)
world = gpd.read_file(world_boundaries)

uk = world[world.name == 'U.K. of Great Britain and Northern Ireland'] 

uk=uk.to_crs(crs_mtr)

uk.plot(ax=ax,alpha=0.1)
plt.title('Unweighted k-means')

In [ ]:
kmeans = KMeans(n_clusters = 3, max_iter=1000, init ='k-means++')
lat_long = df[['lon','lat']]
lot_size = df['coef']
weighted_kmeans_clusters = kmeans.fit(lat_long, sample_weight = lot_size) # Compute k-means clustering.
df['cluster_label'] = kmeans.predict(lat_long, sample_weight = lot_size)
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels = df['cluster_label'] # Labels of each point
ax=df.plot.scatter(y = 'lat', x = 'lon', c=labels, s=50, cmap='viridis')
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)


uk.plot(ax=ax,alpha=0.1)
plt.title('Weighted k-means clustering based on {}'.format(var),fontsize=12, fontweight='bold')

In [ ]:
# Get the people counter data

national_trail_count=prepare_counter_data(ne_strava_data_file,cnt_ct_off_yr_ne)


# Get the people counter data

north_downs_count=prepare_counter_data(ndw_strava_data_file,cnt_ct_off_yr_nd)

# North Downs Way has no data for 2020

df_count=pd.concat([national_trail_count,north_downs_count],axis=1)#.dropna()

df_count_smooth_no_na=df_count


df_count_smooth_no_na.columns=[x.replace("  "," ").replace(" ","_") for x in df_count_smooth_no_na.columns]

#Data wrangling on counter data
df_count_smooth_no_na=df_count_smooth_no_na.T.stack().reset_index()

df_count_smooth_no_na.rename(columns={'level_0':'site',0:'people_count'},inplace=True)

df_count_smooth_no_na['Date']=df_count_smooth_no_na['Date'].dt.to_period('M').astype(str)

df_count_smooth_no_na

# Get the weather data prepared in separate notebooks

weather_df_all_sites=pd.read_pickle(data_folder+'weather_df.pkl')

weather_df_all_sites['Date']=weather_df_all_sites['Date'].astype(str)


# Merge weather data and counter data
df_count_smooth_no_na=df_count_smooth_no_na.merge(weather_df_all_sites,on=['site','Date'],how='inner')

df_count_smooth_no_na['year']=df_count_smooth_no_na['Date'].apply(lambda x: int(x.split('-')[0]))

df_count_smooth_no_na['Date']=df_count_smooth_no_na['Date'].apply(lambda x: int(x.split('-')[1]))


df_count_smooth_no_na['season']=df_count_smooth_no_na['Date'].apply(lambda x : get_season(x))


df_count_smooth_no_na=df_count_smooth_no_na.sort_values(by=['site']).reset_index(drop=True)

df_count_smooth_no_na.rename(columns={'Date':'Month'},inplace=True)

df_count_smooth_no_na['Month']=df_count_smooth_no_na['Month'].map(dict(zip(range(1,13),\
                                                                           [calendar.month_abbr[x]\
                                                                            for x in range(1,13)])))

df_count_smooth_no_na.sample(5)


# Get the site names

data=df_count_smooth_no_na.copy()



data=pd.read_pickle(data_folder+'complete_dataset.pkl')
data.rename(columns={'people_counter_data':'people_count'},inplace=True)

data['year']=data.Date.apply(lambda x: int(x.split('-')[0]))

data=data.dropna()

data['Date']=data['Date'].apply(lambda x: int(x.split('-')[1]))


data['season']=data['Date'].apply(lambda x : get_season(x))


data=data.sort_values(by=['site']).reset_index(drop=True)

data.rename(columns={'Date':'Month'},inplace=True)

data['Month']=data['Month'].map(dict(zip(range(1,13),[calendar.month_abbr[x] for x in range(1,13)])))